In [102]:
%pip install boto3
%pip install sagemaker
%pip install scikit-learn
%pip install pandas
%pip install numpy


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [103]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd

region = 'eu-central-1'
session = boto3.session.Session(profile_name='AWSAdministratorAccess', region_name=region)
print(session)
sm_session = sagemaker.Session(boto_session=session)
print(sm_session)
bucket = 'nadig-sagemaker-tst'
print('Using bucket ' + bucket)

df = pd.read_csv('train.csv')
df.head()
features = list(df.columns)
labels = features.pop(-1)

Session(region_name='eu-central-1')
Using bucket nadig-sagemaker-tst


In [104]:
x = df[features]
y = df[labels]

In [105]:
x.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [106]:
y.head()

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [107]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1700, 20) (300, 20) (1700,) (300,)


In [108]:
trainX = pd.DataFrame(X_train)
trainX[labels] = y_train

testX = pd.DataFrame(X_test)
testX[labels] = y_test

In [109]:
trainX.to_csv('train-V-1.csv', index=False)
testX.to_csv('test-V-1.csv', index=False)

In [110]:
sk_prefix = "sagemaker/nadig-test/sklearncontainer"
trainpath = sm_session.upload_data(path = "train-V-1.csv", bucket = bucket, key_prefix = sk_prefix)
testpath = sm_session.upload_data(path = "test-V-1.csv", bucket = bucket, key_prefix = sk_prefix)

In [111]:
trainpath, testpath

('s3://nadig-sagemaker-tst/sagemaker/nadig-test/sklearncontainer/train-V-1.csv',
 's3://nadig-sagemaker-tst/sagemaker/nadig-test/sklearncontainer/test-V-1.csv')

In [132]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO
import argparse
import os
import pandas as pd
import numpy as np

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

if __name__ == "__main__":
    print('[INFO] Extracting arguments')
    parser = argparse.ArgumentParser()

    parser.add_argument('--n_estimators', type=int, default=100)
    parser.add_argument('--random_state', type=int, default=0)

    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test', type=str, default=os.environ.get('SM_CHANNEL_TEST'))
    parser.add_argument('--train-file', type=str, default='train-V-1.csv')
    parser.add_argument('--test-file', type=str, default='test-V-1.csv')

    args = parser.parse_args()

    print('SKLEARN Version: ', sklearn.__version__)
    print('Joblib version: ', joblib.__version__)

    print('[INFO] Reading data')
    print()

    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    label = features.pop(-1)

    print('Building training and testing datasets \n')

    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]


    model = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state)
    model.fit(X_train, y_train)

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, model_path)

    print('Model has been saved to {} \n'.format(model_path))

    y_pred_test = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred_test)
    test_report = classification_report(y_test, y_pred_test)

    print('\nTest Accuracy: \n', test_accuracy)
    print('\nTest Report: \n', test_report)

Overwriting script.py


In [133]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point='script.py',
    role='arn:aws:iam::911899431626:role/service-role/AmazonSageMaker-ExecutionRole-20230505T113694',
    instance_count=1,
    instance_type='ml.m5.large',
    framework_version=FRAMEWORK_VERSION,
    max_wait=7200,
    max_run=3600,
    use_spot_instances=True,
    base_job_name='nadig-test-custom',
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0
    },
    sagemaker_session=sm_session
)

In [134]:
sklearn_estimator.fit({'train': trainpath, 'test': testpath}, wait=True)

INFO:sagemaker:Creating training-job with name: nadig-test-custom-2024-04-30-17-15-18-283


2024-04-30 17:15:18 Starting - Starting the training job...
2024-04-30 17:15:34 Starting - Preparing the instances for training...
2024-04-30 17:16:17 Downloading - Downloading the training image...
2024-04-30 17:16:37 Training - Training image download completed. Training in progress...2024-04-30 17:16:56,253 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-04-30 17:16:56,256 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-04-30 17:16:56,296 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-04-30 17:16:56,461 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-04-30 17:16:56,473 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-04-30 17:16:56,485 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-04-30 17:16:56,493 sagemaker-training-toolkit INFO     Invoking user

In [135]:
from sagemaker.sklearn.model import SKLearnModel 

from time import gmtime, strftime

model_name = 'nadig-test-custom-model-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    model_data=sklearn_estimator.model_data,
    role='arn:aws:iam::911899431626:role/service-role/AmazonSageMaker-ExecutionRole-20230505T113694',
    entry_point='script.py',
    framework_version=FRAMEWORK_VERSION,
    sagemaker_session=sm_session
)

In [136]:
model

In [137]:
endpoint_name = 'nadig-test-custom-endpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    endpoint_name=endpoint_name
)

Endpoint name: nadig-test-custom-endpoint-2024-04-30-17-30-18


INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2024-04-30-17-30-19-263
INFO:sagemaker:Creating endpoint-config with name nadig-test-custom-endpoint-2024-04-30-17-30-18
INFO:sagemaker:Creating endpoint with name nadig-test-custom-endpoint-2024-04-30-17-30-18


-----!